In [ ]:
#IMPORTS put all here
import random
import matplotlib.pyplot as plt

from datetime import datetime

#tf imports for reading file
import numpy as np
import os
import PIL


import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow.keras import layers

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization
from tensorflow import keras


from tensorflow.keras.regularizers import l2

#extras
print(tf.__version__)
import pathlib
import shutil

In [ ]:
#path to image dataset
spec_path = 'content/spectrograms6sec'

genre_list=os.listdir(spec_path)
genre_list


In [ ]:
#resets the experiment
#%rm -rf content/spectrograms6secnew

In [ ]:
# os.mkdir('content/spectrograms6secnew')
# dest_path='content/spectrograms6secnew'

# for genre in genre_list:
#   os.mkdir(f'content/spectrograms6secnew/{genre}')



# #no of samples per class
# n=3000
# cnt=0

# random.seed(123)

# for genre in genre_list:
#   genre_path=os.path.join(spec_path,genre)
#   dest_genre_path=os.path.join(dest_path,genre)

#   #shuffling and selecting 3000 samples
#   genre_songs=os.listdir(genre_path)
#   random.shuffle(genre_songs)
#   selected_spec=genre_songs[:n]
  
#   for song in selected_spec:
#       source=os.path.join(genre_path,song)
#       dest=os.path.join(dest_genre_path,song)
#       d= shutil.copyfile(source, dest)




In [ ]:
dest_path='content/spectrograms6secnew'
data_dir=pathlib.Path(dest_path)
data_dir

In [ ]:
#adding dataset
batch_size=64
image_height=100
image_width=200


In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(image_height, image_width),
  batch_size=batch_size)

In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(image_height, image_width),
  batch_size=batch_size)


In [ ]:
class_names = train_ds.class_names
print(class_names)

import matplotlib.pyplot as plt

plt.figure(figsize=(45, 15))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")


In [ ]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

In [ ]:
normalization_layer = tf.keras.layers.Rescaling(1./255)
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
%load_ext tensorboard

# TESTING LEARNING RATES

logs/scalars/20221107-221910/validation -0.3
logs/scalars/20221107-231918/validation -0.1
logs/scalars/20221108-001905/validation -0.03
logs/scalars/20221108-011800/validation -0.01
logs/scalars/20221108-021755/validation -0.003
logs/scalars/20221108-031741/validation -0.001
logs/scalars/20221108-041732/validation -0.0003
logs/scalars/20221108-051723/validation -0.0001


```
num_classes = len(class_names)
model = Sequential([
  layers.Rescaling(1./255, input_shape=(image_height, image_width, 3)),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(8, 4, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes,kernel_regularizer=l2(0.1),activation = "linear")
])

#hyperparams
opt = keras.optimizers.Adam(learning_rate=0.3)



model.compile(optimizer=opt,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.summary()

epochs=10

logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    verbose=1,
    callbacks=[tensorboard_callback],
    )
```
```
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
=================================================================
 rescaling_1 (Rescaling)     (None, 100, 200, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 100, 200, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 100, 200, 16)      4624      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 50, 100, 16)      0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 50, 100, 8)        2056      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 25, 50, 8)        0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 10000)             0         
                                                                 
 dense (Dense)               (None, 128)               1280128   
                                                                 
 dense_1 (Dense)             (None, 10)                1290      
                                                                 
=================================================================
Total params: 1,288,994
Trainable params: 1,288,994
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
375/375 [==============================] - 381s 1s/step - loss: 58.4071 - accuracy: 0.0990 - val_loss: 12.3220 - val_accuracy: 0.0982
Epoch 2/10
375/375 [==============================] - 360s 960ms/step - loss: 7.6695 - accuracy: 0.0986 - val_loss: 5.1379 - val_accuracy: 0.0982
Epoch 3/10
375/375 [==============================] - 359s 958ms/step - loss: 4.0421 - accuracy: 0.0986 - val_loss: 3.2170 - val_accuracy: 0.0982
Epoch 4/10
375/375 [==============================] - 359s 957ms/step - loss: 2.8428 - accuracy: 0.0986 - val_loss: 2.5564 - val_accuracy: 0.0982
Epoch 5/10
375/375 [==============================] - 358s 953ms/step - loss: 2.4569 - accuracy: 0.0989 - val_loss: 2.3626 - val_accuracy: 0.0982
Epoch 6/10
375/375 [==============================] - 359s 957ms/step - loss: 2.3537 - accuracy: 0.0989 - val_loss: 2.3194 - val_accuracy: 0.0982
Epoch 7/10
375/375 [==============================] - 357s 952ms/step - loss: 2.3363 - accuracy: 0.0989 - val_loss: 2.3140 - val_accuracy: 0.0982
Epoch 8/10
375/375 [==============================] - 359s 957ms/step - loss: 2.3314 - accuracy: 0.0989 - val_loss: 2.3160 - val_accuracy: 0.0982
Epoch 9/10
375/375 [==============================] - 358s 954ms/step - loss: 2.3328 - accuracy: 0.0989 - val_loss: 2.3149 - val_accuracy: 0.0982
Epoch 10/10
375/375 [==============================] - 359s 958ms/step - loss: 2.3310 - accuracy: 0.0989 - val_loss: 2.3126 - val_accuracy: 0.0982

```


```
num_classes = len(class_names)
model = Sequential([
  layers.Rescaling(1./255, input_shape=(image_height, image_width, 3)),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(8, 4, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes,kernel_regularizer=l2(0.1),activation = "linear")
])

#hyperparams
opt = keras.optimizers.Adam(learning_rate=0.1)



model.compile(optimizer=opt,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.summary()

epochs=10

logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    verbose=1,
    callbacks=[tensorboard_callback],
    )
```
```
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
=================================================================
 rescaling_2 (Rescaling)     (None, 100, 200, 3)       0         
                                                                 
 conv2d_3 (Conv2D)           (None, 100, 200, 32)      896       
                                                                 
 conv2d_4 (Conv2D)           (None, 100, 200, 16)      4624      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 50, 100, 16)      0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 50, 100, 8)        2056      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 25, 50, 8)        0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 10000)             0         
                                                                 
 dense_2 (Dense)             (None, 128)               1280128   
                                                                 
 dense_3 (Dense)             (None, 10)                1290      
                                                                 
=================================================================
Total params: 1,288,994
Trainable params: 1,288,994
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
375/375 [==============================] - 360s 958ms/step - loss: 29.8500 - accuracy: 0.0995 - val_loss: 9.7327 - val_accuracy: 0.1022
Epoch 2/10
375/375 [==============================] - 358s 955ms/step - loss: 8.0908 - accuracy: 0.0985 - val_loss: 6.5925 - val_accuracy: 0.1022
Epoch 3/10
375/375 [==============================] - 360s 960ms/step - loss: 5.4973 - accuracy: 0.0977 - val_loss: 4.5701 - val_accuracy: 0.1022
Epoch 4/10
375/375 [==============================] - 359s 957ms/step - loss: 3.9830 - accuracy: 0.0982 - val_loss: 3.5058 - val_accuracy: 0.1022
Epoch 5/10
375/375 [==============================] - 359s 956ms/step - loss: 3.2334 - accuracy: 0.0986 - val_loss: 3.0100 - val_accuracy: 0.1022
Epoch 6/10
375/375 [==============================] - 358s 954ms/step - loss: 2.8887 - accuracy: 0.0988 - val_loss: 2.7788 - val_accuracy: 0.1022
Epoch 7/10
375/375 [==============================] - 359s 956ms/step - loss: 2.7168 - accuracy: 0.0988 - val_loss: 2.6492 - val_accuracy: 0.1022
Epoch 8/10
375/375 [==============================] - 358s 953ms/step - loss: 2.6086 - accuracy: 0.0988 - val_loss: 2.5567 - val_accuracy: 0.1022
Epoch 9/10
375/375 [==============================] - 359s 957ms/step - loss: 2.5254 - accuracy: 0.0988 - val_loss: 2.4817 - val_accuracy: 0.1022
Epoch 10/10
375/375 [==============================] - 359s 957ms/step - loss: 2.4577 - accuracy: 0.0988 - val_loss: 2.4212 - val_accuracy: 0.1022

```

```
num_classes = len(class_names)
model = Sequential([
  layers.Rescaling(1./255, input_shape=(image_height, image_width, 3)),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(8, 4, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes,kernel_regularizer=l2(0.1),activation = "linear")
])

#hyperparams
opt = keras.optimizers.Adam(learning_rate=0.03)



model.compile(optimizer=opt,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.summary()

epochs=10

logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    verbose=1,
    callbacks=[tensorboard_callback],
    )
```
```
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
=================================================================
 rescaling_3 (Rescaling)     (None, 100, 200, 3)       0         
                                                                 
 conv2d_6 (Conv2D)           (None, 100, 200, 32)      896       
                                                                 
 conv2d_7 (Conv2D)           (None, 100, 200, 16)      4624      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 50, 100, 16)      0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 50, 100, 8)        2056      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 25, 50, 8)        0         
 2D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 10000)             0         
                                                                 
 dense_4 (Dense)             (None, 128)               1280128   
                                                                 
 dense_5 (Dense)             (None, 10)                1290      
                                                                 
=================================================================
Total params: 1,288,994
Trainable params: 1,288,994
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
375/375 [==============================] - 354s 943ms/step - loss: 2.4425 - accuracy: 0.0964 - val_loss: 2.3057 - val_accuracy: 0.1037
Epoch 2/10
375/375 [==============================] - 355s 948ms/step - loss: 2.3064 - accuracy: 0.0953 - val_loss: 2.3049 - val_accuracy: 0.1037
Epoch 3/10
375/375 [==============================] - 351s 936ms/step - loss: 2.3061 - accuracy: 0.0946 - val_loss: 2.3048 - val_accuracy: 0.1037
Epoch 4/10
375/375 [==============================] - 355s 945ms/step - loss: 2.3059 - accuracy: 0.0957 - val_loss: 2.3046 - val_accuracy: 0.1037
Epoch 5/10
375/375 [==============================] - 353s 940ms/step - loss: 2.3058 - accuracy: 0.0967 - val_loss: 2.3044 - val_accuracy: 0.1037
Epoch 6/10
375/375 [==============================] - 355s 946ms/step - loss: 2.3057 - accuracy: 0.0961 - val_loss: 2.3044 - val_accuracy: 0.1037
Epoch 7/10
375/375 [==============================] - 352s 939ms/step - loss: 2.3057 - accuracy: 0.0962 - val_loss: 2.3044 - val_accuracy: 0.1037
Epoch 8/10
375/375 [==============================] - 354s 944ms/step - loss: 2.3057 - accuracy: 0.0962 - val_loss: 2.3044 - val_accuracy: 0.1037
Epoch 9/10
375/375 [==============================] - 353s 941ms/step - loss: 2.3057 - accuracy: 0.0962 - val_loss: 2.3044 - val_accuracy: 0.1037
Epoch 10/10
375/375 [==============================] - 354s 943ms/step - loss: 2.3057 - accuracy: 0.0962 - val_loss: 2.3044 - val_accuracy: 0.1037

```

```
num_classes = len(class_names)
model = Sequential([
  layers.Rescaling(1./255, input_shape=(image_height, image_width, 3)),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(8, 4, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes,kernel_regularizer=l2(0.1),activation = "linear")
])

#hyperparams
opt = keras.optimizers.Adam(learning_rate=0.01)



model.compile(optimizer=opt,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.summary()

epochs=10

logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    verbose=1,
    callbacks=[tensorboard_callback],
    )
```
```
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
=================================================================
 rescaling_4 (Rescaling)     (None, 100, 200, 3)       0         
                                                                 
 conv2d_9 (Conv2D)           (None, 100, 200, 32)      896       
                                                                 
 conv2d_10 (Conv2D)          (None, 100, 200, 16)      4624      
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 50, 100, 16)      0         
 2D)                                                             
                                                                 
 conv2d_11 (Conv2D)          (None, 50, 100, 8)        2056      
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 25, 50, 8)        0         
 2D)                                                             
                                                                 
 flatten_3 (Flatten)         (None, 10000)             0         
                                                                 
 dense_6 (Dense)             (None, 128)               1280128   
                                                                 
 dense_7 (Dense)             (None, 10)                1290      
                                                                 
=================================================================
Total params: 1,288,994
Trainable params: 1,288,994
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
375/375 [==============================] - 356s 950ms/step - loss: 2.3977 - accuracy: 0.0984 - val_loss: 2.3038 - val_accuracy: 0.0980
Epoch 2/10
375/375 [==============================] - 354s 944ms/step - loss: 2.3039 - accuracy: 0.0986 - val_loss: 2.3034 - val_accuracy: 0.0980
Epoch 3/10
375/375 [==============================] - 361s 964ms/step - loss: 2.3038 - accuracy: 0.0991 - val_loss: 2.3034 - val_accuracy: 0.0980
Epoch 4/10
375/375 [==============================] - 360s 960ms/step - loss: 2.3038 - accuracy: 0.0992 - val_loss: 2.3033 - val_accuracy: 0.0980
Epoch 5/10
375/375 [==============================] - 364s 970ms/step - loss: 2.3038 - accuracy: 0.0989 - val_loss: 2.3033 - val_accuracy: 0.0980
Epoch 6/10
375/375 [==============================] - 359s 957ms/step - loss: 2.3038 - accuracy: 0.0990 - val_loss: 2.3033 - val_accuracy: 0.0980
Epoch 7/10
375/375 [==============================] - 362s 964ms/step - loss: 2.3038 - accuracy: 0.0993 - val_loss: 2.3033 - val_accuracy: 0.0980
Epoch 8/10
375/375 [==============================] - 360s 959ms/step - loss: 2.3038 - accuracy: 0.0990 - val_loss: 2.3033 - val_accuracy: 0.0980
Epoch 9/10
375/375 [==============================] - 359s 958ms/step - loss: 2.3038 - accuracy: 0.0992 - val_loss: 2.3033 - val_accuracy: 0.0980
Epoch 10/10
375/375 [==============================] - 360s 960ms/step - loss: 2.3038 - accuracy: 0.0989 - val_loss: 2.3033 - val_accuracy: 0.0980

```

```
num_classes = len(class_names)
model = Sequential([
  layers.Rescaling(1./255, input_shape=(image_height, image_width, 3)),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(8, 4, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes,kernel_regularizer=l2(0.1),activation = "linear")
])

#hyperparams
opt = keras.optimizers.Adam(learning_rate=0.003)



model.compile(optimizer=opt,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.summary()

epochs=10

logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    verbose=1,
    callbacks=[tensorboard_callback],
    )
```
```
Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
=================================================================
 rescaling_5 (Rescaling)     (None, 100, 200, 3)       0         
                                                                 
 conv2d_12 (Conv2D)          (None, 100, 200, 32)      896       
                                                                 
 conv2d_13 (Conv2D)          (None, 100, 200, 16)      4624      
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 50, 100, 16)      0         
 2D)                                                             
                                                                 
 conv2d_14 (Conv2D)          (None, 50, 100, 8)        2056      
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 25, 50, 8)        0         
 2D)                                                             
                                                                 
 flatten_4 (Flatten)         (None, 10000)             0         
                                                                 
 dense_8 (Dense)             (None, 128)               1280128   
                                                                 
 dense_9 (Dense)             (None, 10)                1290      
                                                                 
=================================================================
Total params: 1,288,994
Trainable params: 1,288,994
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
375/375 [==============================] - 360s 959ms/step - loss: 2.1252 - accuracy: 0.2685 - val_loss: 1.7957 - val_accuracy: 0.3875
Epoch 2/10
375/375 [==============================] - 360s 959ms/step - loss: 1.6317 - accuracy: 0.4310 - val_loss: 1.5269 - val_accuracy: 0.4760
Epoch 3/10
375/375 [==============================] - 359s 958ms/step - loss: 1.5035 - accuracy: 0.4798 - val_loss: 1.5015 - val_accuracy: 0.4850
Epoch 4/10
375/375 [==============================] - 358s 955ms/step - loss: 1.5576 - accuracy: 0.4659 - val_loss: 1.4656 - val_accuracy: 0.4910
Epoch 5/10
375/375 [==============================] - 358s 954ms/step - loss: 1.3678 - accuracy: 0.5357 - val_loss: 1.4228 - val_accuracy: 0.5130
Epoch 6/10
375/375 [==============================] - 361s 962ms/step - loss: 1.2600 - accuracy: 0.5745 - val_loss: 1.4866 - val_accuracy: 0.5018
Epoch 7/10
375/375 [==============================] - 358s 954ms/step - loss: 1.1662 - accuracy: 0.6093 - val_loss: 1.5191 - val_accuracy: 0.5035
Epoch 8/10
375/375 [==============================] - 358s 955ms/step - loss: 1.0831 - accuracy: 0.6395 - val_loss: 1.5529 - val_accuracy: 0.5113
Epoch 9/10
375/375 [==============================] - 357s 951ms/step - loss: 1.0005 - accuracy: 0.6688 - val_loss: 1.5741 - val_accuracy: 0.5168
Epoch 10/10
375/375 [==============================] - 359s 956ms/step - loss: 0.9443 - accuracy: 0.6915 - val_loss: 1.6907 - val_accuracy: 0.5100

```

```

num_classes = len(class_names)
model = Sequential([
  layers.Rescaling(1./255, input_shape=(image_height, image_width, 3)),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(8, 4, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes,kernel_regularizer=l2(0.1),activation = "linear")
])

#hyperparams
opt = keras.optimizers.Adam(learning_rate=0.001)



model.compile(optimizer=opt,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.summary()

epochs=10

logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    verbose=1,
    callbacks=[tensorboard_callback],
    )
```

```
Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
=================================================================
 rescaling_6 (Rescaling)     (None, 100, 200, 3)       0         
                                                                 
 conv2d_15 (Conv2D)          (None, 100, 200, 32)      896       
                                                                 
 conv2d_16 (Conv2D)          (None, 100, 200, 16)      4624      
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 50, 100, 16)      0         
 g2D)                                                            
                                                                 
 conv2d_17 (Conv2D)          (None, 50, 100, 8)        2056      
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 25, 50, 8)        0         
 g2D)                                                            
                                                                 
 flatten_5 (Flatten)         (None, 10000)             0         
                                                                 
 dense_10 (Dense)            (None, 128)               1280128   
                                                                 
 dense_11 (Dense)            (None, 10)                1290      
                                                                 
=================================================================
Total params: 1,288,994
Trainable params: 1,288,994
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
375/375 [==============================] - 360s 959ms/step - loss: 2.2036 - accuracy: 0.3553 - val_loss: 1.5372 - val_accuracy: 0.4793
Epoch 2/10
375/375 [==============================] - 361s 962ms/step - loss: 1.4216 - accuracy: 0.5250 - val_loss: 1.3399 - val_accuracy: 0.5493
Epoch 3/10
375/375 [==============================] - 357s 952ms/step - loss: 1.2238 - accuracy: 0.5894 - val_loss: 1.2159 - val_accuracy: 0.5923
Epoch 4/10
375/375 [==============================] - 360s 960ms/step - loss: 1.0849 - accuracy: 0.6390 - val_loss: 1.1919 - val_accuracy: 0.6010
Epoch 5/10
375/375 [==============================] - 359s 956ms/step - loss: 0.9770 - accuracy: 0.6769 - val_loss: 1.1524 - val_accuracy: 0.6137
Epoch 6/10
375/375 [==============================] - 360s 959ms/step - loss: 0.8837 - accuracy: 0.7119 - val_loss: 1.1650 - val_accuracy: 0.6130
Epoch 7/10
375/375 [==============================] - 357s 953ms/step - loss: 0.7955 - accuracy: 0.7458 - val_loss: 1.2113 - val_accuracy: 0.6073
Epoch 8/10
375/375 [==============================] - 358s 956ms/step - loss: 0.7120 - accuracy: 0.7733 - val_loss: 1.2687 - val_accuracy: 0.5982
Epoch 9/10
375/375 [==============================] - 359s 959ms/step - loss: 0.6490 - accuracy: 0.7945 - val_loss: 1.3395 - val_accuracy: 0.5990
Epoch 10/10
375/375 [==============================] - 359s 958ms/step - loss: 0.6092 - accuracy: 0.8043 - val_loss: 1.3789 - val_accuracy: 0.5992

```

```
num_classes = len(class_names)
model = Sequential([
  layers.Rescaling(1./255, input_shape=(image_height, image_width, 3)),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(8, 4, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes,kernel_regularizer=l2(0.1),activation = "linear")
])

#hyperparams
opt = keras.optimizers.Adam(learning_rate=0.0003)



model.compile(optimizer=opt,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.summary()

epochs=10

logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    verbose=1,
    callbacks=[tensorboard_callback],
    )
```
```
Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
=================================================================
 rescaling_7 (Rescaling)     (None, 100, 200, 3)       0         
                                                                 
 conv2d_18 (Conv2D)          (None, 100, 200, 32)      896       
                                                                 
 conv2d_19 (Conv2D)          (None, 100, 200, 16)      4624      
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 50, 100, 16)      0         
 g2D)                                                            
                                                                 
 conv2d_20 (Conv2D)          (None, 50, 100, 8)        2056      
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 25, 50, 8)        0         
 g2D)                                                            
                                                                 
 flatten_6 (Flatten)         (None, 10000)             0         
                                                                 
 dense_12 (Dense)            (None, 128)               1280128   
                                                                 
 dense_13 (Dense)            (None, 10)                1290      
                                                                 
=================================================================
Total params: 1,288,994
Trainable params: 1,288,994
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
375/375 [==============================] - 359s 956ms/step - loss: 2.9556 - accuracy: 0.3053 - val_loss: 2.1315 - val_accuracy: 0.4298
Epoch 2/10
375/375 [==============================] - 359s 957ms/step - loss: 1.8066 - accuracy: 0.4777 - val_loss: 1.5961 - val_accuracy: 0.5232
Epoch 3/10
375/375 [==============================] - 359s 958ms/step - loss: 1.4929 - accuracy: 0.5420 - val_loss: 1.4192 - val_accuracy: 0.5663
Epoch 4/10
375/375 [==============================] - 358s 956ms/step - loss: 1.3604 - accuracy: 0.5783 - val_loss: 1.3356 - val_accuracy: 0.5855
Epoch 5/10
375/375 [==============================] - 359s 957ms/step - loss: 1.2702 - accuracy: 0.6041 - val_loss: 1.2847 - val_accuracy: 0.5983
Epoch 6/10
375/375 [==============================] - 360s 960ms/step - loss: 1.1908 - accuracy: 0.6274 - val_loss: 1.2618 - val_accuracy: 0.6037
Epoch 7/10
375/375 [==============================] - 359s 957ms/step - loss: 1.1246 - accuracy: 0.6479 - val_loss: 1.2373 - val_accuracy: 0.6045
Epoch 8/10
375/375 [==============================] - 359s 958ms/step - loss: 1.0668 - accuracy: 0.6651 - val_loss: 1.2233 - val_accuracy: 0.6088
Epoch 9/10
375/375 [==============================] - 359s 958ms/step - loss: 1.0108 - accuracy: 0.6845 - val_loss: 1.2099 - val_accuracy: 0.6145
Epoch 10/10
375/375 [==============================] - 359s 957ms/step - loss: 0.9582 - accuracy: 0.6993 - val_loss: 1.1972 - val_accuracy: 0.6223

```

```
num_classes = len(class_names)
model = Sequential([
  layers.Rescaling(1./255, input_shape=(image_height, image_width, 3)),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(8, 4, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes,kernel_regularizer=l2(0.1),activation = "linear")
])

#hyperparams
opt = keras.optimizers.Adam(learning_rate=0.0001)



model.compile(optimizer=opt,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.summary()

epochs=10

logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    verbose=1,
    callbacks=[tensorboard_callback],
    )
```
```
Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
=================================================================
 rescaling_8 (Rescaling)     (None, 100, 200, 3)       0         
                                                                 
 conv2d_21 (Conv2D)          (None, 100, 200, 32)      896       
                                                                 
 conv2d_22 (Conv2D)          (None, 100, 200, 16)      4624      
                                                                 
 max_pooling2d_14 (MaxPoolin  (None, 50, 100, 16)      0         
 g2D)                                                            
                                                                 
 conv2d_23 (Conv2D)          (None, 50, 100, 8)        2056      
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 25, 50, 8)        0         
 g2D)                                                            
                                                                 
 flatten_7 (Flatten)         (None, 10000)             0         
                                                                 
 dense_14 (Dense)            (None, 128)               1280128   
                                                                 
 dense_15 (Dense)            (None, 10)                1290      
                                                                 
=================================================================
Total params: 1,288,994
Trainable params: 1,288,994
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
375/375 [==============================] - 361s 961ms/step - loss: 3.3621 - accuracy: 0.3098 - val_loss: 2.7675 - val_accuracy: 0.4415
Epoch 2/10
375/375 [==============================] - 360s 960ms/step - loss: 2.4738 - accuracy: 0.4602 - val_loss: 2.2283 - val_accuracy: 0.4955
Epoch 3/10
375/375 [==============================] - 359s 956ms/step - loss: 2.0568 - accuracy: 0.5067 - val_loss: 1.9171 - val_accuracy: 0.5253
Epoch 4/10
375/375 [==============================] - 361s 963ms/step - loss: 1.8008 - accuracy: 0.5377 - val_loss: 1.7214 - val_accuracy: 0.5478
Epoch 5/10
375/375 [==============================] - 358s 956ms/step - loss: 1.6304 - accuracy: 0.5585 - val_loss: 1.5972 - val_accuracy: 0.5578
Epoch 6/10
375/375 [==============================] - 361s 962ms/step - loss: 1.5106 - accuracy: 0.5782 - val_loss: 1.4987 - val_accuracy: 0.5682
Epoch 7/10
375/375 [==============================] - 362s 966ms/step - loss: 1.4228 - accuracy: 0.5942 - val_loss: 1.4522 - val_accuracy: 0.5767
Epoch 8/10
375/375 [==============================] - 363s 967ms/step - loss: 1.3558 - accuracy: 0.6078 - val_loss: 1.3846 - val_accuracy: 0.5910
Epoch 9/10
375/375 [==============================] - 366s 975ms/step - loss: 1.3001 - accuracy: 0.6202 - val_loss: 1.3533 - val_accuracy: 0.5963
Epoch 10/10
375/375 [==============================] - 363s 969ms/step - loss: 1.2515 - accuracy: 0.6330 - val_loss: 1.3151 - val_accuracy: 0.6035

```

In [ ]:
%load_ext tensorboard